Hola &#x1F600;

Soy **Hesus Garcia**  como "Jesús" pero con H. Sé que puede ser confuso al principio, pero una vez que lo recuerdes, ¡nunca lo olvidarás! &#x1F31D;	. Como revisor de código de Triple-Ten, estoy emocionado de examinar tus proyectos y ayudarte a mejorar tus habilidades en programación. si has cometido algún error, no te preocupes, pues ¡estoy aquí para ayudarte a corregirlo y hacer que tu código brille! &#x1F31F;. Si encuentro algún detalle en tu código, te lo señalaré para que lo corrijas, ya que mi objetivo es ayudarte a prepararte para un ambiente de trabajo real, donde el líder de tu equipo actuaría de la misma manera. Si no puedes solucionar el problema, te proporcionaré más información en la próxima oportunidad. Cuando encuentres un comentario,  **por favor, no los muevas, no los modifiques ni los borres**. 

Revisaré cuidadosamente todas las implementaciones que has realizado para cumplir con los requisitos y te proporcionaré mis comentarios de la siguiente manera:


<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class=“tocSkip”></a>
Si todo está perfecto.
</div>

<div class="alert alert-block alert-warning">
<b>Comentario del revisor</b> <a class=“tocSkip”></a>
Si tu código está bien pero se puede mejorar o hay algún detalle que le hace falta.
</div>

<div class="alert alert-block alert-danger">
<b>Comentario del revisor</b> <a class=“tocSkip”></a>
Si de pronto hace falta algo o existe algún problema con tu código o conclusiones.
</div>

Puedes responderme de esta forma:
<div class="alert alert-block alert-info">
<b>Respuesta del estudiante</b> <a class=“tocSkip”></a>
</div>

</br>

**¡Empecemos!**  &#x1F680;

# Índice <a id='back'></a>

* [Introducción](#intro)
* [Inicialización](#init)
    * [Carga de librerías](#libs)
    * [Carga de datos](#data)
    * [Funciones a usar](#functions)
* [Análisis exploratorio de datos](#eda)
* [Plan de acción](#plan)
* [Preguntas](#questions)
* [Arreglo para modelado](#modelos)
    * [Unión conjunto de datos](#dataset)
        * [Conjunto users](#users)
        * [Conjunto contract](#contract)
        * [Conjunto phone](#phone)
        * [Conjunto internet](#internet)
    * [Preparación de datos](#datos)
        * [Balanceo por sobremuestreo](#upsample)
        * [One Hot Encoding](#ohe)
        * [Encoder type](#label)
        * [Escalado de características númericas](#scaler)
        * [Correlación de características númericas](#corr)
        * [Chi-Cuadrado](#chi)
        * [Boruta](#boruta)
    * [Modelos](#models)
        * [Selección de características para modelos](#select)
        * [LightGBM](#lgbm)
        * [Bosque Aleatorio Clasificatorio](#rfr)
        * [CatBoost](#cbc)
        * [Clasificador XGB](#xgb)
        * [Resumen de resultador por modelos](#results)
* [Conclusiones](#conclusions)

# Proyecto TELECOM

La empresa Interconnect le gustaría pronosticar su tasa de cancelación de clientes. Si se descubre que un susuario o usuaria planea irse, se le ofrecerán códigos promocionales y opciones de planes especiales.
Por lo que solicitaron un análisis del comportamiento para predecir si un cliente está por cancelar el contrato.

## Carga librerías

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import time
import math
import scipy.stats as st
from itertools import combinations
from boruta import BorutaPy
from imblearn.over_sampling import RandomOverSampler


from re import sub

from sklearn.utils import shuffle
from sklearn.model_selection import (train_test_split, 
                                     GridSearchCV, 
                                     cross_val_score)
from sklearn import metrics
from sklearn.metrics import (auc,
                             accuracy_score, 
                             average_precision_score, 
                             f1_score, 
                             mean_squared_error, 
                             precision_recall_curve, 
                             roc_auc_score, 
                             roc_curve)
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier


import warnings
warnings.simplefilter('ignore')


ModuleNotFoundError: No module named 'imblearn'

## Carga de datos

In [ ]:
try:
    users = pd.read_csv('./datasets/personal.csv')
    phone = pd.read_csv('./datasets/phone.csv')
    internet = pd.read_csv('./datasets/internet.csv')
    contract = pd.read_csv('./datasets/contract.csv',parse_dates=['BeginDate'])
    
except:
    users = pd.read_csv('/datasets/final_provider/personal.csv')
    phone = pd.read_csv('/datasets/final_provider/phone.csv')
    internet = pd.read_csv('/datasets/final_provider/internet.csv')
    contract = pd.read_csv('/datasets/final_provider/contract.csv')

## Funciones a usar

Se cargan las funcionas a utilizar en el desarrollo del proyecto.

In [ ]:
#Función para dar formato a nombre de columna de acuerdo con convención
def column_format(c):
    return '_'.join(sub('([A-Z][a-z]+)', r' \1',
                    sub('([A-Z]+)', r' \1',
                    c.replace('-',' '))).split()).lower()

#Función para renombrar las columnas creadas con get_dummies
def bool_columns(df, columns):
    df = pd.get_dummies(data=df, columns=columns, drop_first=True)
    
    for col in df.columns:
        pattern = r'_([A-Z].*)'     # Encuentra el patron que agrega get_dummies
        replace = ''                # El reemplazo es una cadena vacia
        clean_name = sub(pattern=pattern, repl=replace, string=col)
        df = df.rename(columns={col:clean_name})
    return df

In [ ]:
#Funciones de calculo de fechas reales de termino y días activos

def caculate_end_date_by_year(df, last_year=2020, last_month=2, add_n_years=None, add_n_months=None):
    
    if (df['begin_date'].year == last_year) & (add_n_years):
        return df['begin_date'] + pd.DateOffset(years=add_n_years)
    
    else:
        return pd.to_datetime(f'{last_year-1}-{df["begin_date"].month}-01') + pd.DateOffset(years=add_n_years)

def real_end_date(df):
    if pd.isna(df['end_date']):
        type_contract = df['type']

        
        if (type_contract == 'Month-to-month') & (df['begin_date'].month == 2):
            return df['begin_date'] + pd.DateOffset(months=1)
        
        elif (type_contract == 'Month-to-month') & (df['begin_date'].month == 1):
            return pd.to_datetime('2020-02-01')
        
        
        elif type_contract == 'One year':
            return caculate_end_date_by_year(df, add_n_years=1)
        else:
            return caculate_end_date_by_year(df, add_n_years=2)
    else:
        return df['end_date']
    
def active_days(df):
   
    end_active_day = df['real_end_date']
    
    if end_active_day > pd.to_datetime('2020-01-01'):
        end_active_day = pd.to_datetime('2020-01-01')
    return (end_active_day - df['begin_date']).days


In [ ]:
#Función de parámetros AUC-ROC y exactitud
def metricas(model, train_features, train_target, test_features, test_target, proba=True):
    
    eval_stats = {}
    
    fig, axs = plt.subplots(1, 3, figsize=(20, 6)) 
    
    for type, features, target in (('train', train_features, train_target), ('test', test_features, test_target)):
        
        eval_stats[type] = {}
    
        pred_target = model.predict(features)
        pred_proba = model.predict_proba(features)[:, 1]

        # F1
        f1_thresholds = np.arange(0, 1.01, 0.05)
        f1_scores = [f1_score(target, pred_proba>=threshold) for threshold in f1_thresholds]
        
        # ROC-AUC
        fpr, tpr, roc_thresholds = roc_curve(target, pred_proba)
        roc_auc = roc_auc_score(target, pred_proba)    
        eval_stats[type]['ROC AUC'] = roc_auc

        # PRC
        precision, recall, pr_thresholds = precision_recall_curve(target, pred_proba)
        aps = average_precision_score(target, pred_proba)
        eval_stats[type]['APS'] = aps
        
        if type == 'train':
            color = 'blue'
        else:
            color = 'green'

        # Valor F1
        ax = axs[0]
        max_f1_score_idx = np.argmax(f1_scores)
        ax.plot(f1_thresholds, f1_scores, color=color, label=f'{type}, max={f1_scores[max_f1_score_idx]:.2f} @ {f1_thresholds[max_f1_score_idx]:.2f}')
        # establecer cruces para algunos umbrales        
        for threshold in (0.2, 0.4, 0.5, 0.6, 0.8):
            closest_value_idx = np.argmin(np.abs(f1_thresholds-threshold))
            marker_color = 'orange' if threshold != 0.5 else 'red'
            ax.plot(f1_thresholds[closest_value_idx], f1_scores[closest_value_idx], color=marker_color, marker='X', markersize=7)
        ax.set_xlim([-0.02, 1.02])    
        ax.set_ylim([-0.02, 1.02])
        ax.set_xlabel('threshold')
        ax.set_ylabel('F1')
        ax.legend(loc='lower center')
        ax.set_title(f'Valor F1') 

        # ROC
        ax = axs[1]    
        ax.plot(fpr, tpr, color=color, label=f'{type}, ROC AUC={roc_auc:.2f}')
        # establecer cruces para algunos umbrales        
        for threshold in (0.2, 0.4, 0.5, 0.6, 0.8):
            closest_value_idx = np.argmin(np.abs(roc_thresholds-threshold))
            marker_color = 'orange' if threshold != 0.5 else 'red'            
            ax.plot(fpr[closest_value_idx], tpr[closest_value_idx], color=marker_color, marker='X', markersize=7)
        ax.plot([0, 1], [0, 1], color='grey', linestyle='--')
        ax.set_xlim([-0.02, 1.02])    
        ax.set_ylim([-0.02, 1.02])
        ax.set_xlabel('FPR')
        ax.set_ylabel('TPR')
        ax.legend(loc='lower center')        
        ax.set_title(f'Curva ROC')
        
        # PRC
        ax = axs[2]
        ax.plot(recall, precision, color=color, label=f'{type}, AP={aps:.2f}')
        # establecer cruces para algunos umbrales        
        for threshold in (0.2, 0.4, 0.5, 0.6, 0.8):
            closest_value_idx = np.argmin(np.abs(pr_thresholds-threshold))
            marker_color = 'orange' if threshold != 0.5 else 'red'
            ax.plot(recall[closest_value_idx], precision[closest_value_idx], color=marker_color, marker='X', markersize=7)
        ax.set_xlim([-0.02, 1.02])    
        ax.set_ylim([-0.02, 1.02])
        ax.set_xlabel('recall')
        ax.set_ylabel('precision')
        ax.legend(loc='lower center')
        ax.set_title(f'PRC')        

        eval_stats[type]['Accuracy'] = accuracy_score(target, pred_target)
        eval_stats[type]['F1'] = f1_score(target, pred_target)
    
    df_eval_stats = pd.DataFrame(eval_stats, )
    df_eval_stats = df_eval_stats.round(2)
    df_eval_stats = df_eval_stats.reindex(index=('Accuracy', 'F1', 'APS', 'ROC AUC'))
    
    print(df_eval_stats)
    
    return

In [ ]:
#Clasificación de usuarios activos y no en conjunto de datos contract con valores booleanos

def active(df):
    return pd.isna(df['end_date'])

## Análisis exploratorio de datos

In [ ]:
users.columns

In [ ]:
users.columns = [column_format(column) for column in users.columns]

In [ ]:
users.info()

Las columnas no parece que tengan ningún problema con tipo de dato, la columna `senior_citizen` se toma como númerica por la clasificación que tiene de la información, 1 para adultos mayores y 0 para los que no lo son. Se mantendrá el conjunto de datos sin cambios

In [ ]:
users.head()

In [ ]:
phone.columns

In [ ]:
phone.columns = [column_format(column) for column in phone.columns]

In [ ]:
phone.info()

No hay nada por hacer en este conjunto de datos

In [ ]:
phone.head()

In [ ]:
internet.columns

In [ ]:
internet.columns = [column_format(column) for column in internet.columns]

In [ ]:
internet.info()

In [ ]:
internet.head()

In [ ]:
contract.columns

In [ ]:
contract.columns = [column_format(column) for column in contract.columns]

In [ ]:
contract.info()

La columna `total_charges` tiene tipo de objeto, se buscarán los valores mínimos para verificar si existe algún problema con los datos.

In [ ]:
contract.sort_values(by='total_charges')

Hay 11 usuarios que iniciaron su contrato en Febrero, por lo que aún no se les genera cargo, por eso aparecen la columna `total_charges` vacias y el tipo de la columna se mantiene com objeto, se cambiará este valor por 0, y se harán de tipo float para su tratamiento y análisis.

In [ ]:
contract['total_charges'] = contract['total_charges'].replace(' ', '0').astype(float)

In [ ]:
contract.info()

La columna end_date es de tipo objeto, se convertirá en tipo Datetime.

In [ ]:
contract['end_date'] = pd.to_datetime(contract['end_date'], errors='coerce')

In [ ]:
contract.info()

In [ ]:
contract.sample(20)

In [ ]:
contract.describe()

Se creará una nueva columna para tener una separación entre los usuarios activos y los inactivos tomando como referencia si tiene una fecha la columna `end_date` o no. Devolviendo un booleano.

In [ ]:
contract['is_active'] = contract.apply(active, axis=1).astype(float)
contract.sample(10)

In [ ]:
contract.groupby('type')['is_active'].sum() / contract.groupby('type')['is_active'].count() *100

De acuerdo con los valores obtenido arriba, las personas con contrato a dos años son lo que siguen activos en un 97% mientras que los usuarios con contrato mes a mes son los que menos usuarios tenemos.

Buscaremos los años en los que se ha tenido mayor cantidad de usuarios por tipo de contrato, así como en que meses hay mayor demanda de los servicios. Para esto dividiremos las fechas de inicio

In [ ]:
contract['begin_date'] = pd.to_datetime(contract['begin_date'])
contract['begin_year'] = contract['begin_date'].dt.year
contract['begin_month'] = contract['begin_date'].dt.month
contract['end_year'] = contract['end_date'].dt.year
contract['end_month'] = contract['end_date'].dt.month
contract.head()

Para la fechas de termino, las calcularemos la fecha real de termino considerando a los que llevan contrato mes a mes que su corte y fin de servicio es un mes despúes.

In [ ]:
contract['real_end_date'] = contract.apply(real_end_date, axis=1)
contract

In [ ]:
contract['active_days'] = contract.apply(active_days, axis=1)
contract.drop(['begin_date','end_date','real_end_date'],axis=1, inplace=True)
contract

In [ ]:
ax = contract.groupby(['begin_year','type']).size().unstack(fill_value=0).plot(kind='bar', figsize=(9,6))
ax.set_xlabel('Año')
ax.set_ylabel('Contratos')
ax.set_title('Nuevos usuarios por año y tipo')
plt.xticks()
plt.legend()
plt.tight_layout()
plt.show()


En 2019 se unieron mas usuarios por contrato mes a mes que en otros años.

In [ ]:
ax = contract.groupby(['end_year','type']).size().unstack(fill_value=0).plot(kind='bar', figsize=(9,6))
ax.set_xlabel('Año')
ax.set_ylabel('Contratos')
ax.set_title('Usuarios inactivos por año y tipo')
plt.xticks()
plt.legend()
plt.tight_layout()
plt.show()

En 2019 los usuarios inactivos aumentaron con contrato de Mes a Mes.

In [ ]:
ax = contract.groupby(['begin_month','type']).size().unstack(fill_value=0).plot(kind='bar', figsize=(9,6))
ax.set_xlabel('Mes')
ax.set_ylabel('Contratos')
ax.set_title('Nuevos usuarios por mes y tipo')
plt.xticks()
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
ax = contract.groupby(['end_month','type']).size().unstack(fill_value=0).plot(kind='bar', figsize=(9,6))
ax.set_xlabel('Mes')
ax.set_ylabel('Contratos')
ax.set_title('Usuarios inactivos por mes y tipo')
plt.xticks()
plt.legend()
plt.tight_layout()
plt.show()

Los usuarios que más han abandonado el servicio son los usuarios de contrato mes a mes, como se vio anteriormente en los memses de Octubre, Noviembre, Diciembre y Enero, siendo los dos últimos años los que han tenido bajas.

Con esta información se hará lo mismo buscando si el tipo de pago influye en el abandono del servicio de los usuarios con contrato mes a mes.

In [ ]:
ax = contract[contract['type']=='Month-to-month'].groupby(['begin_year','payment_method']).size().unstack(fill_value=0).plot(kind='bar', figsize=(9,6))
ax.set_xlabel('Año')
ax.set_ylabel('Contratos')
ax.set_title('Nuevos usuarios Mes a Mes por año y tipo de pago')
plt.xticks()
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
ax = contract[contract['type']=='Month-to-month'].groupby(['end_year','payment_method']).size().unstack(fill_value=0).plot(kind='bar', figsize=(9,6))
ax.set_xlabel('Año')
ax.set_ylabel('Contratos')
ax.set_title('Abandono de usuarios mes a mes por año y tipo de pago')
plt.xticks()
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
ax = contract[contract['type']=='Month-to-month'].groupby(['begin_month','payment_method']).size().unstack(fill_value=0).plot(kind='bar', figsize=(9,6))
ax.set_xlabel('Mes')
ax.set_ylabel('Contratos')
ax.set_title('Nuevos usuarios mes a mes por mes y tipo de pago')
plt.xticks()
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
ax = contract[contract['type']=='Month-to-month'].groupby(['end_month','payment_method']).size().unstack(fill_value=0).plot(kind='bar', figsize=(9,6))
ax.set_xlabel('Mes')
ax.set_ylabel('Contratos')
ax.set_title('Abandono usuarios mes a mes por mes y tipo de pago')
plt.xticks()
plt.legend()
plt.tight_layout()
plt.show()

La tendencia de abandono en los meses de Octubre a Enero se mantiene, mostrando que los usuarios con pago electrónico son los más propensos a dejar los servicios de la compñía.

## Plan de acción

   - Limpieza de datos
       * Se buscarán valores ausentes en cada conjunto de datos dandoles tratamiento.
       * Se dará solución a posibles valores duplicados en los conjuntos de datos.
   - Unión de tablas
       * Se unirán las tablas tomando como columna guía el cutomerID.
   - Entrenamiento de modelos
       - Divisón de dataset.
           * Se divirirá el dataset con todas las columnas en entrenamiento y prueba para los diferentes modelos de clasificación a probar.
       - Entrenamiento y prueba de modelos.
           * Con cada modelo evaluaremos AUC-ROC y exactitud.
       - Ajuste de modelo a usar.
           * Teniendo el modelo con mejores valores, se ajustarán los hiperparametros para obtener mejores resultados con el modelo.
    

## Preguntas

- ¿Se sabe si los usuarios que se dan de baja han regresado con otro tipo de contrato?
- ¿Al cambiar un método de pago se realiza nuevo contrato?
- ¿Adicionar elementos al contrato se genera como uno nuevo desechando el anterior?

## Arreglo para modelado

### Unión de conjunto de datos

### Conjunto `users`

Se buscará preparar el conjunto de datos `users` para poder unirlo al dataset `contract`

In [ ]:
users.info() 

In [ ]:
users.describe()

Convertiremos las columnas `gender, partner y dependents` en dummies.

In [ ]:
users = bool_columns(users, ['gender','partner','dependents'])
users.head()

Ahora la columna `paperless_billing` en *contract* se hará dummie

In [ ]:
contract = bool_columns(contract, ['paperless_billing'])
contract.head()

Unión de `contract` con `users`

In [ ]:
df = pd.merge(contract, users, on='customer_id', how='outer')

In [ ]:
df

## Conjunto `phone`

Al igual que el conjunto `users` se unirá el conjunto `phone` en un solo dataset

In [ ]:
phone.head()

In [ ]:
phone =bool_columns(phone, ['multiple_lines'])
phone.head()

In [ ]:
df = pd.merge(df, phone, on='customer_id', how='outer')
df

### Conjunto `Internet`

Unión de `internet` al set `df` con todas la características a evaluar.

In [ ]:
internet.head()

In [ ]:
internet = bool_columns(internet, 
                          ['online_security',
                                             'online_backup',
                           'internet_service',
                                             'device_protection',
                                             'tech_support',
                                             'streaming_tv',
                                             'streaming_movies'])
internet.head()

In [ ]:
df = pd.merge(df, internet, on='customer_id', how='outer')
df

Se buscaran valores ausentes

In [ ]:
df.isna().sum()

Estos valore se reemplazarán por un `0`

In [ ]:
df.fillna('0', inplace=True)
df.info()

Para realizar el modelado las fechas de inicio y termino del contrato no se utilizarán por lo que se descartarán de nuestro conjunto de datos a trabajar, así como el `customer_id`

In [ ]:
df.drop(['customer_id','begin_year','begin_month','end_year','end_month'],axis=1, inplace=True)
df.info()

## Preparación de datos 

Nuestra variable objetivo es `is_active`, es decir vamos a predecir si un usuario está activo o no con contrato en la compañia dependiendo de las características de su contrato.

In [ ]:
#Se crean los conjuntos de características (features) y de objetico (target)

features = df.drop('is_active',axis=1)
target = df['is_active']

### Balanceo por sobremuestreo

In [ ]:
over_sampler = RandomOverSampler(random_state=12345)
features_upsampled, target_upsampled = over_sampler.fit_resample(features, target)

In [ ]:
features_upsampled

In [ ]:
#División en conjunto de entrenamiento y prueba.
features_train, features_test, target_train, target_test = train_test_split(features_upsampled, 
                                                                            target_upsampled, 
                                                                            test_size=0.3, 
                                                                            random_state=12345)

In [ ]:
features_train.sample()

In [ ]:
features_test.sample()

### One Hot-Encoding

La columna `payment_method` se hará dummie para separar en características los datos y poder evaluar.

In [ ]:
features_train_enc = pd.get_dummies(features_train, columns=['payment_method',])
features_test_enc = pd.get_dummies(features_test, columns=['payment_method'])

In [ ]:
features_train_enc.head()

### Encoder de `type`

A la columna `type` se le aplicará una codificación ya que de momento la tenemos como tipo `str`, esto nos ayudará a mejorar las predicciones del modelo

In [ ]:
label_encoder = LabelEncoder()
features_train_enc['type'] = label_encoder.fit_transform(features_train_enc['type'])
features_test_enc['type'] = label_encoder.fit_transform(features_test_enc['type'])

In [ ]:
features_train_enc.info()

### Escalado de features númericos

Para uniformizar todos los valores númericos de las columnas `type`, `monthly_charges`, `total_charges` y `active_days`, le aplicaremos un escalamiento de minimos y maximos, esto para dar mejor certeza a las predicciones del modelo.

In [ ]:
scaler = MinMaxScaler()
features_train_enc_sc = features_train_enc.copy()
features_test_enc_sc = features_test_enc.copy()
columns_to_scale = ['type','monthly_charges','total_charges','active_days']
features_train_enc_sc[columns_to_scale] = scaler.fit_transform(features_train_enc[columns_to_scale])
features_test_enc_sc[columns_to_scale] = scaler.fit_transform(features_test_enc[columns_to_scale])
features_train_enc_sc

### Correlación de features númericas

In [ ]:
pd.concat([features_train_enc_sc[columns_to_scale], target_train], axis=1).corr()

Correlaciones débiles con *is_active* (valor menor a 0.3)
   * `monthly_charges` tiene una correlación negativa, por lo que no influye en si un usuario se queda con la compañía.
   * `total_charges` su correlación es 0.15, aún sigue siendo baja.


Correlaciones fuertes con *is_active* (valor mayor a 0.3)
   * `active_days` tiene una correlación alta con un valor de 0.37
   * `type` tiene una correlación alta, por lo que el tipo de contrato sigue siendo definitorio para el cliente.

### Chi-Cuadrado

Se usará la prueba de Chi-cuadrado para verificar si las columnas se relacionan.

In [ ]:
for combination in combinations(df.columns,2):
    contingency_table = pd.crosstab(df[combination[0]], df[combination[1]])
    chi2, p, _, _ = st.chi2_contingency(contingency_table)
    alpha = 0.05
    if p > alpha:
        print(f"Table de Contingencia para {combination}:")
        print(contingency_table)
        print("Test Chi-Cuadrado:")
        print("Estadíctico Chi-Cuadrado:", chi2)
        print("Valor p:", p)
        print("No hay asociación entre las variables")
        print(" ")

Gracias a la prueba de Chi-Cuadrado podemos descartar el género ya que no tiene relación con prácticamente ninguna otra característica y no es una variable de peso para nuestro modelo

## Boruta

Utilizamos el método BorutaPy para determinar las características que realmente intervienen en la desceción de los usuarios de la compañía, con esto dejaremos unicamente las columnas necesarias para los entrenamientos y pruebas.

In [ ]:
model = RandomForestClassifier(random_state=12345)

boruta_selector = BorutaPy(estimator=model, n_estimators='auto', verbose=2)

boruta_selector.fit(features_train_enc.values, target_train)

In [ ]:
boruta_features = features_train_enc.columns[boruta_selector.support_]
boruta_features

In [ ]:
features_train_enc.columns[~boruta_selector.support_]

<div class="alert alert-block alert-danger">
<b>Comentario del revisor</b> <a class="tocSkip"></a>
Buen trabajo en la parte inicial y de preprocesamiento tu proyecto.Sin embargo, Para potenciarlo, es fundamental concentrarnos en la selección de características. Esta etapa clave puede influir significativamente en el rendimiento del modelo y notado que no se ha desarrollado.  Comienza con técnicas simples de filtrado, como análisis de correlación y pruebas estadísticas. Luego, considera métodos más avanzados. En algunos cohortes (puede que no sea tu caso)  del bootcamp, se han explorado técnicas como Boruta y SHAP, que ofrecen enfoques sofisticados para identificar las características más relevantes. Revisa tus apuntes de estas sesiones con el tutor para una guía detallada. Implementar una selección de características efectiva puede mejorar la precisión y eficiencia de tus modelos. ¡Sigue adelante!
</div>


## Modelos

### Selección de características para modelos

Con Boruta terminado, filtramos y creamos los datasets que utilizaremos para realizar las predicciones con diferentes modelos de clasificación

In [ ]:
features_train_enc = features_train_enc[boruta_features]
features_test_enc = features_test_enc[boruta_features]
features_train_enc_sc = features_train_enc_sc[boruta_features]
features_test_enc_sc = features_test_enc_sc[boruta_features]
features_train_enc

### LightGBM

Primero calculamos el valor cruzado de validación sin hiperparámetros para cada modelo.

In [ ]:
model = LGBMClassifier()

scores_lgbm = cross_val_score(model,
                        features_train_enc_sc,
                        target_train,
                        scoring='roc_auc').mean()
scores_lgbm

In [ ]:
model = LGBMClassifier()

parameters = {
    'objective': ['binary'],
    'metric':['auc'],
    'learning_rate': [0.1],
    'max_depth': [12,24],
    'reg_lambda': [1],
    'n_estimators': [100,1000],
    'num_leaves': [12,24]
}

grid = GridSearchCV(estimator=model,
                   param_grid=parameters,
                   scoring='roc_auc')

grid.fit(features_train_enc_sc, target_train)

best_param = grid.best_params_
best_model = grid.best_estimator_
best_param

In [ ]:
start_lgbm = time.time()

lgbm = best_model

lgbm.fit(features_train_enc_sc, target_train)

finish_fit_lgbm = time.time()

lgbm_proba = lgbm.predict_proba(features_test_enc_sc)[:,1]
lgbm_pred = lgbm.predict(features_test_enc_sc)

finish_predict_lgbm = time.time()
roc_auc_lgbm = round(roc_auc_score(target_test, lgbm_proba),3)
accuracy_lgbm = round(accuracy_score(target_test, lgbm_pred),3)
lgbmf_time = round(finish_fit_lgbm - start_lgbm,3)
lgbmp_time = round(finish_predict_lgbm - start_lgbm,3)

metricas(lgbm, features_train_enc_sc, target_train, features_test_enc_sc, target_test)

El modelo LightGBM arroja un buen valor AUC-ROC de 0.97 para el conjunto de prueba.

### Bosque Aleatorio Clasificatorio

In [ ]:
model = RandomForestClassifier(random_state=12345)

scores_rfr = cross_val_score(model,
                        features_train_enc_sc,
                        target_train,
                        scoring='roc_auc').mean()
scores_rfr

In [ ]:
parameters = {
    'max_depth':[None,2,4,40],
    'random_state':[12345]
}

grid = GridSearchCV(estimator=model,
                   param_grid=parameters,
                   scoring='roc_auc')

grid.fit(features_train_enc_sc, target_train)

best_param = grid.best_params_
best_model = grid.best_estimator_
best_param

In [ ]:
start_rfr = time.time()

rfr = best_model

rfr.fit(features_train_enc_sc, target_train)

finish_fit_rfr = time.time()

rfr_proba = rfr.predict_proba(features_test_enc_sc)[:,1]
rfr_pred = rfr.predict(features_test_enc_sc)

finish_predict_rfr = time.time()

roc_auc_rfr = round(roc_auc_score(target_test, rfr_proba),3)
accuracy_rfr = round(accuracy_score(target_test, rfr_pred),3)
rfrf_time = round(finish_fit_rfr - start_rfr,3)
rfrp_time = round(finish_predict_rfr - start_rfr,3)

metricas(rfr, features_train_enc_sc, target_train, features_test_enc_sc, target_test)

Al igual que el modelo anterior el AUC-ROC nos arroja un valor 0.97

### CatBoost

In [ ]:
model = CatBoostClassifier(random_state=12345)

scores_cbc = cross_val_score(model,
                        features_train_enc_sc,
                        target_train,
                        scoring='roc_auc').mean()
scores_cbc

In [ ]:
parameters = {
    'learning_rate':[0.1],
    'depth':[6],
    'l2_leaf_reg':[1],
    'iterations':[500],
    'random_state':[12345]
}

grid = GridSearchCV(estimator= model,
                   param_grid = parameters,
                   scoring='roc_auc',
                   cv=5)

grid.fit(features_train_enc_sc, target_train)



best_param = grid.best_params_
best_model = grid.best_estimator_
best_param

In [ ]:
start_cbc = time.time()

cbc = best_model

cbc.fit(features_train_enc_sc, target_train.astype(int))

finish_fit_cbc = time.time()

cbc_pred = cbc.predict(features_test_enc_sc)
cbc_proba = cbc.predict_proba(features_test_enc_sc)[:,1]

finish_predict_cbc = time.time()

roc_auc_cbc = round(roc_auc_score(target_test, cbc_proba),3)
accuracy_cbc = round(accuracy_score(target_test, cbc_pred),3)
cbcf_time = round(finish_fit_cbc - start_cbc,3)
cbcp_time = round(finish_predict_cbc - start_cbc,3)
metricas(cbc, features_train_enc_sc, target_train, features_test_enc_sc, target_test)

El modelo Catboost nos da un valor de 0.96, dando un poco peor modelo que los anteriores

### Clasificador XGB

In [ ]:
model = XGBClassifier(random_state=12345)

scores_xgb = cross_val_score(model,
                        features_train_enc_sc,
                        target_train,
                        scoring='roc_auc').mean()
scores_xgb

In [ ]:
parameters = {
    'max_depth':[3,6],
    'learning_rate':[0.1,0.2],
    'max_leaves':[0,2],
    'max_depth':[24],
    'reg_lambda':[1],
    'random_state':[12345],
    'n_estimators':[10,100,1000]
    
}

grid = GridSearchCV(estimator= model,
                   param_grid = parameters,
                   scoring='roc_auc')

grid.fit(features_train_enc_sc, target_train)

best_param = grid.best_params_
best_model = grid.best_estimator_
best_param

In [ ]:
start_xgb = time.time()

xgb = best_model

xgb.fit(features_train_enc_sc, target_train)

finish_fit_xgb = time.time()

xgb_pred = xgb.predict(features_test_enc_sc)
xgb_proba = xgb.predict_proba(features_test_enc_sc)[:,1]

finish_predict_xgb = time.time()

roc_auc_xgb = round(roc_auc_score(target_test, xgb_proba),3)
accuracy_xgb = round(accuracy_score(target_test, xgb_pred),3)
xgbf_time = round(finish_fit_xgb - start_xgb,3)
xgbp_time = round(finish_predict_xgb - start_xgb,3)

metricas(xgb, features_train_enc_sc, target_train, features_test_enc_sc, target_test)

Al igual que los primeros modelos el valor AUC-ROC nos arroja un 0.97

### Resumen de resultados por modelos

In [ ]:
index = ['LightGBM','RandomForest','CatBoost','XGB']

d = {'ROC-AUC':[roc_auc_lgbm, roc_auc_rfr, roc_auc_cbc, roc_auc_xgb],
     'Accuracy':[accuracy_lgbm, accuracy_rfr, accuracy_cbc, accuracy_xgb],
    'time_fitting':[lgbmf_time, rfrf_time, cbcf_time, xgbf_time],
    'time_predicting':[lgbmp_time, rfrp_time, cbcp_time, xgbp_time],
    'Mean Cross Validation':[scores_lgbm, scores_rfr, scores_cbc, scores_xgb]} 

models = pd.DataFrame(data=d, index=index).sort_values(by='ROC-AUC', ascending=False)
models

<div class="alert alert-block alert-danger">
<b>Comentario del revisor</b> <a class="tocSkip"></a>
Al revisar los resultados de tus modelos, es crucial verificar no solo su rendimiento en el conjunto de entrenamiento sino también en el conjunto de prueba. Un alto rendimiento en el conjunto de entrenamiento combinado con un rendimiento significativamente más bajo en el conjunto de prueba es un indicador clásico de overfitting. Implementar técnicas como la validación cruzada o ajustar la complejidad del modelo (por ejemplo, mediante la regularización) puede ayudar a mitigar este problema. Además, es esencial equilibrar el rendimiento entre el entrenamiento y la prueba para garantizar que el modelo generalice bien a nuevos datos.</div>


## Conclusiones

Los datos que obtuvimos los trabajamos para obtener los valores dummies y poder dejar las características para entrenar los modelos adecuadamente.
Con los valores de las características se codifico la columna `type`, con el tipo de contrato con el que están dados de alta los usuarios de la empresa.

Ya con todas la columnas trabajadas entre booleanos y números buscamos entrenar cada modelo propuesto y obtener las predicciones de clases y las probabilidades de obtener las clases para el calculo de precisión y de AUC-ROC.

Graficamos para cada modelo las gráficas de AUC-ROC y F1 para mostrar el comportamiento del modelo.

El modelo LightGBM, fue el modelo que nos arrojó mejor valor de precisión y de AUC-ROC por lo que podemos usar este modelo para presentarlo a la empresa Telecom.

Dentro de nuestros hallazgos, encontramos que los usuarios que tienen contrato mes a mes son los que más abandonan la compañía, por lo que sugerimos que presten mayor atención a estos usuarios para ofrecerles beneficios dirante un mes para comprobar que ayuda a que los usuarios se queden con la compañía.

<div class="alert alert-block alert-warning">
<b>Comentario del revisor</b> <a class="tocSkip"></a>
<strong>Lamentablemente, no podemos aprobar tu proyecto en este momento.</strong> Es importante que atiendas los siguientes puntos para asegurar que tu análisis cumpla con los estándares requeridos y pueda ser aprobado:<br><br>

- <strong>Overfitting en Modelos:</strong>
  <ul>
    <li>Verifica el rendimiento de tus modelos tanto en el conjunto de entrenamiento como en el de prueba. La validación cruzada y la regularización son técnicas clave para prevenir el overfitting.</li>
    <li>Asegúrate de que haya un equilibrio en el rendimiento entre los conjuntos de entrenamiento y prueba, indicando que tu modelo generaliza correctamente a nuevos datos.</li>
  </ul><br>

- <strong>Selección de Características:</strong>
  <ul>
    <li>Implementa una etapa de selección de características antes de entrenar los modelos. Utiliza técnicas de filtrado básicas, como análisis de correlación, y explora métodos más avanzados (por ejemplo, Boruta y SHAP) para identificar las características más relevantes.</li>
    <li>Esta etapa es crucial para mejorar la precisión de tus modelos y evitar el overfitting.</li>
  </ul><br>

- <strong>Claridad en Markdown:</strong>
  <ul>
    <li>Es esencial mejorar la estructura y claridad de tu presentación en Markdown.Hay varias secciones dónde solo tenemos un título pero no se describe ni se justifica que se está haciendo, por ejemplo, la selección de parametros de modelos.  Utiliza subtítulos claros, explica tus decisiones y organiza tu análisis de manera lógica para facilitar la comprensión.</li>
  </ul><br>

<strong>Te animo a revisar estos aspectos y realizar las correcciones necesarias.</strong> Recuerda que el proceso de aprendizaje está lleno de desafíos y superarlos es parte de tu formación como analista de datos. Estoy seguro de que, con estos ajustes, tu proyecto no solo cumplirá con los requisitos sino que también reflejará la calidad de tu trabajo y tu capacidad para realizar análisis de datos complejos. <strong>¡Ánimo y esperamos tu proyecto con las correcciones!</strong>
</div>


<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a>

¡Impresionante esfuerzo en este proyecto! 🌟 **Este proyecto está listo para ser aprobado.**<br>
Tu habilidad para ejecutar y presentar este trabajo es admirable.<br>
- Gran trabajo con las nueas correciones implementaste la selección de características de forma adecuada. 
- El markdown queda mucho más claro ahora. 
<br>Es un placer reconocer tu dedicación y el análisis detallado que has llevado a cabo. Continúa superándote en tus futuras iniciativas. Confío en que aplicarás este conocimiento de manera efectiva en desafíos futuros, avanzando hacia objetivos aún más ambiciosos.
</div>
